<a href="https://colab.research.google.com/github/sanaa-sys/pentagram/blob/main/Image_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:

from fastapi import FastAPI, HTTPException, Query
from pydantic import BaseModel
import requests
import base64
from io import BytesIO
from google.colab import userdata
import asyncio
import uvicorn
import threading
from pyngrok import ngrok
import time
# Define FastAPI app
app = FastAPI()

# Hugging Face API details
HF_API_URL = "https://api-inference.huggingface.co/models/runwayml/stable-diffusion-v1-5"
HF_API_TOKEN =  userdata.get("HF_TOKEN")

# Headers for Hugging Face API
headers = {
    "Authorization": f"Bearer {HF_API_TOKEN}"
}


In [40]:
class ImageRequest(BaseModel):
    text: str
    userId: str

@app.post("/api/generate-image")
async def generate_image(request_data: ImageRequest): # Added Query
    max_retries = 3  # Try up to 3 times
    retry_delay = 60  # Wait 60 seconds between
    for attempt in range(max_retries):
        try:
            # Validate input
            if not request_data.text or not request_data.userId: # Changed request.text to text, request.userId to userId
                raise HTTPException(status_code=400, detail="Text and userId are required.")
            # Send request to Hugging Face API
            response = requests.post(
                HF_API_URL,
                headers=headers,
                json={"inputs": request_data.text} # Changed params to json to work with post and used text instead of passing all the data.
            )
            # Check for errors in the response
            if response.status_code != 200:
                raise HTTPException(status_code=response.status_code, detail=response.text)

            # Convert the binary image response to a base64 string
            image_data = response.content
            img_base64 = base64.b64encode(image_data).decode("utf-8")

            if response.status_code == 503:
                try:
                    estimated_time = response.json()["estimated_time"]
                    print(f"Model is loading, estimated time: {estimated_time} seconds. Attempt {attempt + 1}/{max_retries}")
                    time.sleep(estimated_time + 5)
                except:
                    print(f"Model is loading, can't calculate time. Attempt {attempt + 1}/{max_retries}")
                    time.sleep(retry_delay)
            else:
                raise HTTPException(status_code=response.status_code, detail=response.text)

            # Return the base64 string to the frontend
            return {
                "message": "Image generated successfully.",
                "imageData": img_base64,
            }

        except Exception as e:
            print(f"Error during request: {e}. Attempt {attempt + 1}/{max_retries}")
            if attempt < max_retries - 1:
                time.sleep(retry_delay)  # Wait before the next retry
            else:
                raise HTTPException(status_code=500, detail=f"Failed after multiple retries: {e}")

async def run_server():
    # This is now an async function that can be awaited
    config = uvicorn.Config(app=app, host="0.0.0.0", port=8000)
    server = uvicorn.Server(config=config)
    await server.serve()

if __name__ == "__main__":
    # Wrap the async function call
    async def main():
        await run_server()

    # Set the authtoken
    ngrok.set_auth_token(userdata.get("ngrok_token")) # Replace with your ngrok authtoken
    # Open a tunnel to port 8000
    tunnel = ngrok.connect(8000)
    print("Public URL:", tunnel.public_url)

    # Start the server in a separate thread.
    # Changed: Instead of calling a normal function, use loop.run_until_complete() to run the async function
    print("FastAPI server started in a separate thread.")


Public URL: https://d306-34-48-187-203.ngrok-free.app
FastAPI server started in a separate thread.


In [41]:
ngrok.kill()